In [1]:
import yfinance as yf
import re
import pandas as pd
from pandas import DataFrame, Series

In [2]:
%pylab inline
from datetime import datetime
#from yahoofinancials import YahooFinancials

Populating the interactive namespace from numpy and matplotlib


In [3]:
SP = yf.Ticker('ES=F')

In [4]:
df = SP.history(period = '7d', interval = '1m')

In [5]:
df.shape

(7497, 7)

In [6]:
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2021-09-08 00:00:00-04:00,4518.75,4518.75,4518.75,4518.75,0,0,0
2021-09-08 00:01:00-04:00,4518.75,4518.75,4518.75,4518.75,70,0,0
2021-09-08 00:02:00-04:00,4518.75,4518.75,4518.75,4518.75,91,0,0
2021-09-08 00:03:00-04:00,4518.75,4518.75,4518.75,4518.75,50,0,0
2021-09-08 00:04:00-04:00,4518.75,4518.75,4518.75,4518.75,21,0,0
...,...,...,...,...,...,...,...
2021-09-15 12:45:00-04:00,4473.00,4473.25,4469.75,4470.50,4900,0,0
2021-09-15 12:46:00-04:00,4470.75,4471.75,4470.00,4471.50,2384,0,0
2021-09-15 12:47:00-04:00,4471.25,4471.25,4470.00,4470.25,832,0,0


In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.layers import LeakyReLU

In [8]:
RNN_df = df[['Close', 'Volume']]
RNN_df

,Close,Volume
Datetime,,
2021-09-08 00:00:00-04:00,4518.75,0
2021-09-08 00:01:00-04:00,4518.75,70
2021-09-08 00:02:00-04:00,4518.75,91
2021-09-08 00:03:00-04:00,4518.75,50
2021-09-08 00:04:00-04:00,4518.75,21
...,...,...
2021-09-15 12:45:00-04:00,4470.50,4900
2021-09-15 12:46:00-04:00,4471.50,2384
2021-09-15 12:47:00-04:00,4470.25,832


In [9]:
#notes for prediction
seq_len = 60
future_period_predict = 2
ratio_to_predict= 'ES' 

#classification target
def classify(current, dollar_change):
    tot_change=0
    for x in RNN_df['dollar_change']:
        tot_change = tot_change + x
        if dollar_change >= 1:
            return 1
        elif dollar_change <= -0.5:
            return 0 
        elif tot_change<0:
            tot_change=0
            continue
        else:
            continue
#shift timeseries down 3 spots for 3 hour prediction
dollar_change = ((RNN_df['Close'].shift(-future_period_predict))-RNN_df['Close'])

#create a 3 hour future column
RNN_df["dollar_change"] = dollar_change
RNN_df['Future'] = RNN_df['Close'].shift(-future_period_predict)
#RNN_df['Future double'] = RNN_df['Close'].shift(-future_period_predict*5)
RNN_df

C:\Users\rudra\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\rudra\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Close,Volume,dollar_change,Future
Datetime,,,,
2021-09-08 00:00:00-04:00,4518.75,0,0.00,4518.75
2021-09-08 00:01:00-04:00,4518.75,70,0.00,4518.75
2021-09-08 00:02:00-04:00,4518.75,91,0.00,4518.75
2021-09-08 00:03:00-04:00,4518.75,50,0.00,4518.75
2021-09-08 00:04:00-04:00,4518.75,21,0.00,4518.75
...,...,...,...,...
2021-09-15 12:45:00-04:00,4470.50,4900,-0.25,4470.25
2021-09-15 12:46:00-04:00,4471.50,2384,-0.25,4471.25
2021-09-15 12:47:00-04:00,4470.25,832,0.25,4470.50


In [10]:
RNN_df=RNN_df.dropna()

#send close and 3d future to classify function
def grab_arguments(s):
    return classify(s[0],s[1])

#create a classification
RNN_df['target'] = RNN_df[['Close','dollar_change']].T.apply(grab_arguments)


C:\Users\rudra\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
RNN_df



,Close,Volume,dollar_change,Future,target
Datetime,,,,,
2021-09-08 00:00:00-04:00,4518.75,0,0.00,4518.75,NaN
2021-09-08 00:01:00-04:00,4518.75,70,0.00,4518.75,NaN
2021-09-08 00:02:00-04:00,4518.75,91,0.00,4518.75,NaN
2021-09-08 00:03:00-04:00,4518.75,50,0.00,4518.75,NaN
2021-09-08 00:04:00-04:00,4518.75,21,0.00,4518.75,NaN
...,...,...,...,...,...
2021-09-15 12:43:00-04:00,4473.75,1898,-3.25,4470.50,0.0
2021-09-15 12:44:00-04:00,4472.75,5856,-1.25,4471.50,0.0
2021-09-15 12:45:00-04:00,4470.50,4900,-0.25,4470.25,NaN


In [12]:

#mask= (RNN_df['target'] == 0) or (RNN_df['target'] == 1)
RNN_df = RNN_df.fillna(0)
RNN_df = RNN_df.T.drop(['Future']).T
RNN_df = RNN_df.T.drop(['dollar_change']).T
#train_set = train_set.T.drop(['Future']).T
#test_set = test_set.T.drop(['Future']).T
#train_set = train_set.T.drop(['dollar_change']).T
#test_set = test_set.T.drop(['dollar_change']).T
RNN_df

,Close,Volume,target
Datetime,,,
2021-09-08 00:00:00-04:00,4518.75,0.0,0.0
2021-09-08 00:01:00-04:00,4518.75,70.0,0.0
2021-09-08 00:02:00-04:00,4518.75,91.0,0.0
2021-09-08 00:03:00-04:00,4518.75,50.0,0.0
2021-09-08 00:04:00-04:00,4518.75,21.0,0.0
...,...,...,...
2021-09-15 12:43:00-04:00,4473.75,1898.0,0.0
2021-09-15 12:44:00-04:00,4472.75,5856.0,0.0
2021-09-15 12:45:00-04:00,4470.50,4900.0,0.0


In [13]:
################################
##split into train and test set
#############################

#make sure index is sorted. probs already was
times = sorted(RNN_df.index)

# last 5% of time as test set
last_10pct = times[-int(0.1*len(times))]
print(last_10pct)


2021-09-14 23:53:00-04:00


In [14]:
# test set is last 10% and beyond
test_set = RNN_df[RNN_df.index >= last_10pct]
test_set
#train set is anything under the last 10%
train_set =RNN_df[(RNN_df.index < last_10pct)]
train_set

,Close,Volume,target
Datetime,,,
2021-09-08 00:00:00-04:00,4518.75,0.0,0.0
2021-09-08 00:01:00-04:00,4518.75,70.0,0.0
2021-09-08 00:02:00-04:00,4518.75,91.0,0.0
2021-09-08 00:03:00-04:00,4518.75,50.0,0.0
2021-09-08 00:04:00-04:00,4518.75,21.0,0.0
...,...,...,...
2021-09-14 23:48:00-04:00,4449.50,10.0,0.0
2021-09-14 23:49:00-04:00,4450.00,5.0,0.0
2021-09-14 23:50:00-04:00,4449.25,21.0,0.0


In [15]:
#Drop the right answer, 3 hour future
#train_set = train_set.T.drop(['Future']).T
#test_set = test_set.T.drop(['Future']).T
#train_set = train_set.T.drop(['dollar_change']).T
#test_set = test_set.T.drop(['dollar_change']).T
len(train_set)
len(test_set)

747

In [16]:
#test_set

In [15]:
## check for infinte values in your data

#train_set['Volume'].preprocessing.scale(train_set['Volume'])
import numpy as np
#check for inf as well as -inf
#mask1 = isinf(-train_set['Volume'])
#train_set[mask1]


#mask = isinf(test_set['Volume'])
#test_set[mask]

# mask = train_set['Volume'] == float(inf)
# train_set[mask]= 0

train_set.loc[(train_set['Volume'] == float("inf")),'Volume']=0
train_set[train_set["Volume"] == 0]

test_set.loc[(test_set['Volume'] == float("inf")),'Volume']=0
test_set[test_set["Volume"] == 0]

# Replacing infinite with nan
#train_set.replace([np.inf, -np.inf], np.nan, inplace=True)
  
# Dropping all the rows with nan values
#train_set.dropna(inplace=True)
  
# Printing df


# Replacing infinite with nan
#test_set.replace([np.inf, -np.inf], np.nan, inplace=True)
  
# Dropping all the rows with nan values
#test_set.dropna(inplace=True)
  
# Printing df
#len(test_set)
#len(train_set)
#find the infinite values to get rid of
# 2021-05-05 13:30:00-04:00
#train_set.T["2021-05-05 13:30:00-04:00"]
#drop that index or indices
#RNN_df=RNN_df[~RNN_df.isin([np.nan, np.inf, -np.inf]).any(1)]
#len(RNN_df)

C:\Users\rudra\Anaconda3\lib\site-packages\pandas\core\indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\rudra\Anaconda3\lib\site-packages\pandas\core\indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
C:\Users\rudra\Anaconda3\lib\site-packages\pandas\core\indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

,Close,Volume,target
Datetime,,,


In [16]:
mask = (test_set['Volume'] == float("inf"))
test_set[mask]

,Close,Volume,target
Datetime,,,


In [17]:
#train_set = train_set.drop('2019-12-02 09:30:00-05:00')
#train_set = train_set.drop('2019-12-26 09:30:00-05:00')
#train_set = train_set.drop('2020-05-14 12:30:00-04:00')
#train_set = train_set.drop('2020-12-28 09:30:00-05:00')
#train_set = train_set.drop('2021-05-05 14:30:00-04:00')
#train_set = train_set.drop('2020-11-30 09:30:00-05:00')

In [18]:
###################
#Preprocessing - scale and normalize
#####################

#import preprocessing
from sklearn import preprocessing

# import deque continual appending to list and maintains list len
from collections import deque

import numpy as np
import random

def preprocess_df(df):
    df.dropna(inplace=True)
    
    for col in df:
        
        if col != "target":
            #normalize with % change 
            df[col] = df[col].pct_change()
            df.dropna(inplace=True)
            #scale the data
            df.loc[(df['Volume'] == float("inf")),'Volume']=0
            #print(df[df[col] == float("inf")])
            df[col]= preprocessing.scale(df[col].values)
        
    
    df.dropna(inplace = True)
    
    sequential_data = []
    #maintain the seq_len
    prev_days = deque(maxlen=seq_len)
    #turns df into a list of lists
    for i in df.values:
        # take each value except for target col in list
        prev_days.append([n for n in i[:-1]])
        #print(len(prev_days))
        if len(prev_days) == seq_len:
            sequential_data.append([np.array(prev_days), i[-1]])
            #print(sequential_data)
    
    random.shuffle(sequential_data)  
    
    #####################
    ### Balance the Data 
    ######################

    buys = []
    sells = []
    for seq, target in sequential_data:
        if target == 0:
            sells.append([seq,target])
        elif target ==1:
            buys.append([seq,target])
     
    random.shuffle(buys)
    random.shuffle(sells)
    
    lower = min(len(buys), len(sells)) 
    buys = buys[:lower]
    sells = sells[:lower]
    #print(sells)
    sequential_data = buys+sells
    
    random.shuffle(sequential_data)
    
    x = []
    y = []
    
    for seq, target in sequential_data:
        x.append(seq)
        y.append(target)
        
    return np.array(x), y 

#preprocess_df(train_set)
train_x, train_y = preprocess_df(train_set)
test_x, test_y = preprocess_df(test_set)

C:\Users\rudra\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\rudra\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\rudra\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:

In [23]:
count = 0
for i in test_y:
    if i == 0:
        count+=1

count

140

In [21]:
RNN_df

,Close,Volume,target
Datetime,,,
2021-09-01 00:00:00-04:00,4532.25,0.0,0.0
2021-09-01 00:01:00-04:00,4532.25,48.0,0.0
2021-09-01 00:02:00-04:00,4532.25,49.0,0.0
2021-09-01 00:03:00-04:00,4532.25,46.0,0.0
2021-09-01 00:04:00-04:00,4532.25,99.0,0.0
...,...,...,...
2021-09-09 12:17:00-04:00,4508.50,4126.0,1.0
2021-09-09 12:18:00-04:00,4508.75,2204.0,1.0
2021-09-09 12:19:00-04:00,4511.25,11427.0,0.0


In [22]:
print(f"train data:{len(train_x)} validation: {len(test_x)}")

train data:1236 validation: 248


In [23]:
print(f"Dont buys:{train_y.count(0)}, buys:{train_y.count(1)}")

Dont buys:618, buys:618


In [24]:
print(f"VALIDATION Dont buys: {test_y.count(0)}, buys: {test_y.count(1)}")

VALIDATION Dont buys: 124, buys: 124


In [25]:

epochs = 100

batch_size = 64


In [26]:
# check to make sure training and test set are numpy arrays
test_y = np.asarray(test_y)
train_y = np.asarray(train_y)

In [35]:
from datetime import datetime

def build_model():
    model = Sequential()
    model.add(LSTM(128, input_shape= (train_x.shape[1:]), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(LSTM(128, input_shape= (train_x.shape[1:]), return_sequences=True))
    model.add(Dropout(0.1))
    model.add(BatchNormalization())

    model.add(LSTM(128, input_shape= (train_x.shape[1:])))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(tf.keras.layers.Flatten())
    #dense layer activation rectified linear

    model.add(Dense(32, activation ='relu'))
    model.add(Dropout(0.2))

    #output layer
    model.add(Dense(2, activation = 'softmax'))
    return model

def train(lr, decay, opt_name, epochs = 100, batch_size = 64):

    model = build_model()
    if opt_name.lower() == "sgd":
        opt = tf.keras.optimizers.SGD(lr= lr, decay=decay)
    else:
        opt = tf.keras.optimizers.Adam(lr= lr, decay=decay)
    
    model.compile(loss='sparse_categorical_crossentropy',
                optimizer = opt,
                metrics= ['accuracy'])

    # tensorboard = TensorBoard(log_dir = f'log/{name}')
    now = datetime.now().strftime("%m0%d-%Y-%H-%M-%S")
    name = f"lr--{lr}--decay--{decay}-opt-{opt_name}--datetime-{now}"

    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = f'log/{name}')

    history = model.fit(
                train_x, train_y,
                batch_size=batch_size,
                epochs=epochs,
                validation_data=(test_x, test_y),
                callbacks=[tensorboard_callback])
    # model.save('')

In [36]:
train(.01, 1e-3, "adam")

Epoch 1/100
20/20 [==============================] - 23s 659ms/step - loss: 1.0730 - accuracy: 0.5400 - val_loss: 0.7184 - val_accuracy: 0.5040
Epoch 2/100
20/20 [==============================] - 8s 427ms/step - loss: 0.7194 - accuracy: 0.5669 - val_loss: 0.6958 - val_accuracy: 0.5000
Epoch 3/100
20/20 [==============================] - 10s 530ms/step - loss: 0.6550 - accuracy: 0.6456 - val_loss: 0.8722 - val_accuracy: 0.5000
Epoch 4/100
20/20 [==============================] - 9s 443ms/step - loss: 0.6236 - accuracy: 0.6570 - val_loss: 0.8411 - val_accuracy: 0.5000
Epoch 5/100
20/20 [==============================] - 8s 426ms/step - loss: 0.6125 - accuracy: 0.6771 - val_loss: 0.7567 - val_accuracy: 0.5000
Epoch 6/100
20/20 [==============================] - 8s 391ms/step - loss: 0.6297 - accuracy: 0.6650 - val_loss: 0.8667 - val_accuracy: 0.5000
Epoch 7/100
20/20 [==============================] - 8s 406ms/step - loss: 0.6202 - accuracy: 0.6644 - val_loss: 0.8198 - val_accuracy: 0.50